In [8]:
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
# from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)
from datasets import Dataset
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Lowercasing the text except for the placeholders

In [16]:
# Function to lowercase text except for placeholders
def lowercase_placeholders(text):
    # Step 1: Find placeholders (entities)
    placeholders = re.findall(r'\{\{.*?\}\}', text)
    
    # Step 2: Replace placeholders with temporary markers
    temp_markers = {placeholder: f"PLACEHOLDER_{i}" for i, placeholder in enumerate(placeholders)}
    
    for placeholder, temp_marker in temp_markers.items():
        text = text.replace(placeholder, temp_marker)
    
    # Step 3: Lowercase the text
    lowercased_text = text.lower()

    # Step 4: Restore placeholders
    for placeholder, temp_marker in temp_markers.items():
        lowercased_text = lowercased_text.replace(temp_marker.lower(), placeholder)
    
    return lowercased_text

df = pd.read_csv('sampled_dataset.csv')

#convert instruction to lowercase
df['instruction'] = df['instruction'].apply(lowercase_placeholders)

#convert response to lowercase
df['response'] = df['response'].apply(lowercase_placeholders)

print(df)

print("asdasdsa")

      flags                                        instruction  category  \
0       BLZ           i am tring to see the early exit penalty    CANCEL   
1        BK                             check cancellation fee    CANCEL   
2       BEL  i do not know how to check the cancellation ch...    CANCEL   
3       BLM             i want help to see the early exit fees    CANCEL   
4        BL                            checking early exit fee    CANCEL   
...     ...                                                ...       ...   
2845    BLW  information about correcting the damn shipping...  SHIPPING   
2846    BLM              i have troubles correcting my address  SHIPPING   
2847  BLMQZ  i have errors trying to edit the shippikng add...  SHIPPING   
2848    BIL                      how can i modify the address?  SHIPPING   
2849    BEL    why i cannot enter a new shipping address valid  SHIPPING   

                       intent  \
0      check_cancellation_fee   
1      check_cancella

In [ ]:
# # Get the unique categories
# categories = df['category'].unique()
# print(len(categories))

# # Get the unique intents
# intents = df['intent'].unique()
# print(len(intents))

# # Select the first 3 categories
# selected_categories = categories[:3]

# # Filter the DataFrame to include only the selected categories
# filtered_df = df[df['category'].isin(selected_categories)]

# # Define the number of samples per category
# samples_per_category = 950  # Adjust this number as needed

# # Group by category and sample
# sampled_df = filtered_df.groupby('category').apply(lambda x: x.sample(n=samples_per_category, random_state=42)).reset_index(drop=True)

# # Check the distribution of the sampled data
# print(sampled_df['category'].value_counts())

# # Save the sampled dataset to a new CSV file
# sampled_df.to_csv('sampled_dataset.csv', index=False)

11
27
category
CANCEL      950
ORDER       950
SHIPPING    950
Name: count, dtype: int64


C:\Users\Admin\AppData\Local\Temp\ipykernel_7596\1423447274.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = filtered_df.groupby('category').apply(lambda x: x.sample(n=samples_per_category, random_state=42)).reset_index(drop=True)


Check the data in the sampled dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Convert to list of dictionaries for multi-task classification
data = [{"text": row['instruction'], "category": row['category'], "intent": row['intent']} for _, row in df.iterrows()]

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Split train data into train and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Optional: Encode both the category and intent labels to numerical values
category_encoder = {category: idx for idx, category in enumerate(df['category'].unique())}
intent_encoder = {intent: idx for idx, intent in enumerate(df['intent'].unique())}

# Encode labels in both category and intent
train_data = [{"text": entry['text'], 
               "category": category_encoder[entry['category']], 
               "intent": intent_encoder[entry['intent']]} for entry in train_data]

val_data = [{"text": entry['text'], 
             "category": category_encoder[entry['category']], 
             "intent": intent_encoder[entry['intent']]} for entry in val_data]

test_data = [{"text": entry['text'], 
              "category": category_encoder[entry['category']], 
              "intent": intent_encoder[entry['intent']]} for entry in test_data]

print(data[0])
print(category_encoder)
print(intent_encoder)

print(train_data[0])
print(val_data[0])
print(test_data[20])
print("sadasdasdas")

{'text': 'i am tring to see the early exit penalty', 'category': 'CANCEL', 'intent': 'check_cancellation_fee'}
{'CANCEL': 0, 'ORDER': 1, 'SHIPPING': 2}
{'check_cancellation_fee': 0, 'change_order': 1, 'place_order': 2, 'cancel_order': 3, 'track_order': 4, 'change_shipping_address': 5, 'set_up_shipping_address': 6}
{'text': 'i forgot to include an item in purchase {{Order Number}}', 'category': 1, 'intent': 1}
{'text': 'assistance checking the termination fees', 'category': 0, 'intent': 0}
{'text': 'want help modifying my address', 'category': 2, 'intent': 5}


#Multi task gpt2 classification collator for intent and category 

In [ ]:
class MultiTaskGpt2ClassificationCollator:
    def __init__(self, use_tokenizer, category_encoder, intent_encoder, max_sequence_len=None):
        self.use_tokenizer = use_tokenizer
        self.max_sequence_len = max_sequence_len or use_tokenizer.model_max_length
        self.category_encoder = category_encoder
        self.intent_encoder = intent_encoder
    
    def __call__(self, batch):
        # Extract texts, category labels, and intent labels from the batch
        texts = [item['text'] for item in batch]
        category_labels = [item['category'] for item in batch]
        intent_labels = [item['intent'] for item in batch]
        
        # Tokenize the texts
        inputs = self.use_tokenizer(
            texts, 
            padding=True, 
            truncation=True, 
            max_length=self.max_sequence_len, 
            return_tensors="pt"
        )
        
        # Convert category and intent labels to tensors
        inputs.update({
            'category_labels': torch.tensor(category_labels),
            'intent_labels': torch.tensor(intent_labels)
        })
        
        return inputs

Dataloader


In [ ]:
#dataloaders

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Initialize the collator for multi-task classification
collator = MultiTaskGpt2ClassificationCollator(
    use_tokenizer=tokenizer, 
    category_encoder=category_encoder,
    intent_encoder=intent_encoder,
    max_sequence_len=512
)

# Create DataLoader for both train and validation sets
train_dataloader = DataLoader(train_data, batch_size=32, collate_fn=collator)
test_dataloader = DataLoader(test_data, batch_size=32, collate_fn=collator)
val_dataloader = DataLoader(val_data, batch_size=32, collate_fn=collator)

In [19]:
import torch
print(torch.__version__)  # This will print the version of PyTorch
print(torch.cuda.is_available())  # Should return True if GPU is available
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0)) 

print("asdas")

2.5.1+cu118
True
NVIDIA GeForce RTX 3050 Laptop GPU
asdas
